In [2]:
# Importing requird python packages
# Path to hsmpy package
HSMPY_PATH = r'\\CHCFPP01\Guest\MRajabi\Python Scripts'
import sys, os, arcpy, csv, json, math
sys.path.append(HSMPY_PATH)
import hsmpy3
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from datetime import datetime
from IPython.display import Image
from IPython.core.display import HTML 
import subprocess 
import webbrowser
import time
import copy
import shutil
from time import gmtime, strftime

In [3]:
# Defining Parameteres
print(strftime("%Y-%m-%d %H:%M:%S"))
Years = range(2005,2017)
MaxOpenProcesses = 15

Project_DIR  = r'\\CHCFPP01\Proj\ILDOT\650511SAFETYPROGRAM\4_WorkData\WO19\HSIP_Tracking_Tool'
Project_GDB  = Project_DIR + '\\2.Project_GDB.gdb'
ContractsDir = Project_DIR + '\\4.Contracts_AnalysisFiles'
DocumentsDir = Project_DIR + '\\5.Documents'
PythonDir    = Project_DIR + '\\6.Python_Codes'
ReceivedDir  = Project_DIR + '\\7.ReceivedData'
XML_Dir      = ReceivedDir + '\\HSIP Candidate Applications'
QC_EFK_Dir   = Project_DIR + '\\8.Data_Collection_EFK'

# IRIS Dataset
IRISPath = r'\\CHCFPP01\Proj\ILDOT\650511SAFETYPROGRAM\7_BaseData\Master_BaseFiles_Data\5.0_Roadway_Layer\IRIS'
IRISPath = r'C:\Users\mr068144\Downloads\IRIS\IL_IRIS.gdb'
IRIS_Route = {y:os.path.join(IRISPath,'HWY'+str(y)+'_route') for y in Years}
IRIS_Tab   = {y:os.path.join(IRISPath,'HWY'+str(y)+'_table_HCurve') for y in Years}
IRIS_Tab[2012] = IRIS_Tab[2013]

# Intersections Dataset
IntPG = r'C:\Users\mr068144\Downloads\6.0_Intersection_Layer\IRIS_Ints.gdb\Int2014_PG'
Intersections = {year:IntPG for year in Years}

#Crash Dataset
CrashData  = {y:os.path.abspath(r'C:\Users\mr068144\Downloads\IL Crash Data\IL_Crash.gdb\\' +
                          'CrashExtract_' + str(y) + '_GIS') for y in Years}

Fields = ['AADT',
          'O_SHD1_TYP','O_SHD1_WTH','O_SHD2_TYP','O_SHD2_WTH',
          'I_SHD1_TYP','I_SHD1_WTH','I_SHD2_TYP','I_SHD2_WTH',
          'LNS','LN_WTH','MED_TYP','MED_WTH','SURF_WTH','URBAN','PG','Radius','CurveLen']
#Fields = ['AADT','URBAN','PG']
RouteID = 'INVENTORY'
BMP = 'BEG_STA'
EMP = 'END_STA'

# Geocoded projects
HSIP_Int  = Project_GDB + '\\WPPS_Int'
HSIP_Seg  = Project_GDB + '\\WPPS_Seg'

HSIP_Path    = ReceivedDir + '\\HSIP_AllApprovedProjects.xlsm'
WPPS_Ob_Path = ReceivedDir + '\\HSIP Contract and Obligation Data.xlsx'
WPPS_Pr_Path = ReceivedDir + '\\HSIP Project Programmed Data.xlsx'
SPF_Data     = ReceivedDir  + '\\SPF Coefficients.xlsx'

2018-11-08 16:13:10


In [106]:
# Reading Input Data
print(strftime("%Y-%m-%d %H:%M:%S"))
XML_DF     = hsmpy3.il.HSIP_DF_From_XML(XML_Dir)

HSIP_DF    = hsmpy3.il.CreateHSIPDataFrame(HSIP_Path,Years)
HSIP_DF = HSIP_DF[[(i in XML_DF.HSIPID) for i in HSIP_DF.index]]

WPPS_Ob_DF = hsmpy3.il.HSIP_DF_WPPS_Ob(WPPS_Ob_Path,HSIP_DF,XML_DF)
WPPS_Pr_DF = hsmpy3.il.HSIP_DF_WPPS_Pr(WPPS_Pr_Path,HSIP_DF,XML_DF)

HSIP_Joined = hsmpy3.il.Joined_HSIP_DF(XML_DF,HSIP_DF,WPPS_Ob_DF,WPPS_Pr_DF)

Cont_DF = hsmpy3.il.Joined_Contract_DF(HSIP_Joined,WPPS_Ob_DF,WPPS_Pr_DF,ReceivedDir,Years)

Loc_DF = hsmpy3.il.Joined_Loc_DF(WPPS_Pr_DF)
Loc_DF = Loc_DF.drop_duplicates(subset=['Year', 'INVENTORY', 'BEG_STA', 'END_STA', 'ContNum'])

SPF_DF = pd.read_excel(SPF_Data,sheetname='Summary')
SPF_DF['PGNumber'] = [int(s.split('PeerGroup ')[1][:2]) for s in SPF_DF.PG]

PN = list(set(Loc_DF['ContNum']))
PN.sort()

Len_DF = pd.DataFrame(Loc_DF.groupby('ContNum')['Length'].aggregate(np.sum))
Cont_DF = pd.concat([Cont_DF,Len_DF],axis=1)
print(strftime("%Y-%m-%d %H:%M:%S"))

2018-08-17 10:29:22
2018-08-17 10:33:09


In [137]:
Cont_DF = hsmpy3.il.Joined_Contract_DF(HSIP_Joined,WPPS_Ob_DF,WPPS_Pr_DF,ReceivedDir,Years)
Cont_DF = pd.concat([Cont_DF,Len_DF],axis=1)

In [6]:
# Geocoding WPPS Contracts
print(strftime("%Y-%m-%d %H:%M:%S"))
Main_GDB = AnalysisDir + '\\WPPS_Geocoded.gdb'
DF = pd.merge(Loc_DF,Cont_DF,on='ContNum')
#DF = DF.loc[[i for i,r in DF.iterrows() if r.Status=='C' and r.CompletionDate<datetime(2015,1,1) and r.AwardDate>=datetime(2007,1,1) and r.CompletionAmount>0]]
DF = DF.sort_values(by = ['AwardDate'])
SegFC = hsmpy3.il.GeocodeWPPS(DF,Main_GDB,Project_GDB + '\\WPPS_Seg',IRIS_Route)

2018-06-18 15:24:14


In [140]:
# Add WPPS Location
df = hsmpy3.common.FCtoDF(Project_GDB + '\\WPPS_Seg',True)
df.index = list(df.ContNum)
df = df[['Shape','INVENTORY']]

C2I_DF_M = pd.DataFrame(columns=['ContNum','WPPSLoc','INVENTORY'])
C2I_DF_M['ContNum'] = list(set(df.index))
C2I_DF_M['WPPSLoc'] = [[] for i in list(C2I_DF_M.index)]
C2I_DF_M['INVENTORY'] = [[] for i in list(C2I_DF_M.index)]
C2I_DF_M.index = C2I_DF_M['ContNum']
for i,r in C2I_DF_M.iterrows():
    C2I_DF_M.loc[i,'WPPSLoc'].extend([s for s in df[df.index==i]['Shape']])
    C2I_DF_M.loc[i,'INVENTORY'].extend([s for s in df[df.index==i]['INVENTORY']])
C2I_DF_M['NumSegs'] = [len(l) for l in C2I_DF_M.WPPSLoc]
Cont_DF['WPPSLoc'] = C2I_DF_M.loc[Cont_DF.index,'WPPSLoc']
if 'INVENTORY' in list(Cont_DF):
    if Cont_DF[Cont_DF.INVENTORY.isnull()].index.isin(C2I_DF_M.index).any():
        Cont_DF.loc[Cont_DF[Cont_DF.INVENTORY.isnull()].index,'INVENTORY'] = C2I_DF_M.loc[Cont_DF[Cont_DF.INVENTORY.isnull()].index,'INVENTORY']
else:    
    Cont_DF['INVENTORY'] = C2I_DF_M.loc[Cont_DF.index,'INVENTORY']

In [141]:
# Add EFK Location
print(strftime("%Y-%m-%d %H:%M:%S"))
Int_DF = hsmpy3.common.FCtoDF(QC_EFK_Dir + '\\QC_GDB.gdb\\T2014TrafficControl',readGeometry=True,selectedFields=[])
Int_DF['IntID2'] = list(Int_DF.index)

df = pd.read_excel(QC_EFK_Dir + '\\QC_Contracts_EFKMoen_recieved08072018.xlsx',skiprows=1)[['ContNum','Intersection OBJECTID','What is the Current/updated Location Type? (Roadway, Intersection) ']]
df.columns = ['ContNum','IntID','LocType']
df.index = df.ContNum
df = df[['IntID','LocType']]
df['IntID2'] = [hsmpy3.common.ConvertType(i,'LONG') for i in df.IntID]

C2I_DF = df[(df.LocType=='Intersection') & (~pd.isnull(df.IntID2))][['IntID2']]
C2I_DF = C2I_DF.join(Int_DF,on='IntID2',lsuffix='Int')[['IntID2','Shape']]

C2I_DF_M = pd.DataFrame(columns=['ContNum','EFKLoc'])
C2I_DF_M['ContNum'] = list(set(C2I_DF.index))
C2I_DF_M['EFKLoc'] = [[] for i in list(C2I_DF_M.index)]
C2I_DF_M.index = C2I_DF_M['ContNum']
for i,r in C2I_DF_M.iterrows():
    C2I_DF_M.loc[i,'EFKLoc'].extend([s for s in C2I_DF[C2I_DF.index==i]['Shape']])
C2I_DF_M['NumPoints'] = [len(l) for l in C2I_DF_M.EFKLoc]
Cont_DF['EFKLoc'] = C2I_DF_M.loc[Cont_DF.index,'EFKLoc']

2018-08-17 10:54:05


In [142]:
# Create 1 to 1 projects conversion dictionary
print(strftime("%Y-%m-%d %H:%M:%S"))
HSIP_Joined['NumOfCont'] = [len(s.split(';')) for s in HSIP_Joined.ContNums]
Cont_DF['NumOfHSIP'] = [len(str(s).split(';')) for s in Cont_DF.HSIPIDs]
NumOfJoinedCont = []
NumOfCompJoinedCont = []
for i,r in Cont_DF.iterrows():
    cont_l = []
    hsip_ids = str(r.HSIPIDs).split(';')
    for hsip in hsip_ids:
        cont_l.extend(HSIP_Joined.loc[hsip]['ContNums'].split(';'))
        cont_l.extend(HSIP_Joined.loc[hsip]['ContNums'].split(';'))
        s = ''
        for cont in HSIP_Joined.loc[hsip]['ContNums'].split(';'):
            cont_l.extend(HSIP_Joined.loc[hsip]['ContNums'].split(';'))
    cont_l = list(set(cont_l))
    NumOfJoinedCont.append(';'.join(cont_l))
Cont_DF['Joined_Conts'] = NumOfJoinedCont
Cont_DF['NumOfJoinedConts'] = [len(s.split(';')) for s in Cont_DF.Joined_Conts]
ConvDict = Cont_DF[Cont_DF.NumOfJoinedConts==1][['HSIPIDs']].to_dict()['HSIPIDs']
ConvDictInv = {} 
for k in ConvDict.keys():
    for h in ConvDict[k].split(';'):
        ConvDictInv.update({int(h):str(k)})
print(len(ConvDict))
print(len(ConvDictInv))

2018-08-17 10:54:38
615
669


In [144]:
# Add HSIP Loc Data
print(strftime("%Y-%m-%d %H:%M:%S"))
df = hsmpy3.common.FCtoDF(Project_GDB + '\\HSIP_Int',True)
df = df[df.HSIP_ID.isin(list(ConvDictInv.keys()))]
df['Shape'] = [s.projectAs(hsmpy3.common.NAD1983IL) for s in df.Shape]
df['ContNum'] = [ConvDictInv[h] for h in df.HSIP_ID]
df.index = df['ContNum']
C2I_DF_M = pd.DataFrame(columns=['ContNum','HSIPIntLoc'])
C2I_DF_M['ContNum'] = list(set(df.index))
C2I_DF_M['HSIPIntLoc'] = [[] for i in list(C2I_DF_M.index)]
C2I_DF_M.index = C2I_DF_M['ContNum']
for i,r in C2I_DF_M.iterrows():
    C2I_DF_M.loc[i,'HSIPIntLoc'].extend([s for s in df[df.index==i]['Shape']])
Cont_DF['HSIPIntLoc'] = C2I_DF_M.loc[Cont_DF.index,'HSIPIntLoc']

df = hsmpy3.common.FCtoDF(Project_GDB + '\\HSIP_Seg',True)
df = df[df.HSIP_ID.isin(list(ConvDictInv.keys()))]
df['ContNum'] = [ConvDictInv[h] for h in df.HSIP_ID]
df.index = df['ContNum']
C2I_DF_M = pd.DataFrame(columns=['ContNum','HSIPSegLoc','INVENTORY'])
C2I_DF_M['ContNum'] = list(set(df.index))
C2I_DF_M['HSIPSegLoc'] = [[] for i in list(C2I_DF_M.index)]
C2I_DF_M['INVENTORY'] = [[] for i in list(C2I_DF_M.index)]
C2I_DF_M.index = C2I_DF_M['ContNum']
for i,r in C2I_DF_M.iterrows():
    C2I_DF_M.loc[i,'HSIPSegLoc'].extend([s for s in df[df.index==i]['Shape']])
    C2I_DF_M.loc[i,'INVENTORY'].extend(['' for s in df[df.index==i]['Shape']])
Cont_DF['HSIPSegLoc'] = C2I_DF_M.loc[Cont_DF.index,'HSIPSegLoc']
if 'INVENTORY' in list(Cont_DF):
    if Cont_DF[Cont_DF.INVENTORY.isnull()].index.isin(C2I_DF_M.index).any():
        Cont_DF.loc[Cont_DF[Cont_DF.INVENTORY.isnull()].index,'INVENTORY'] = C2I_DF_M.loc[Cont_DF[Cont_DF.INVENTORY.isnull()].index,'INVENTORY']
else:    
    Cont_DF['INVENTORY'] = C2I_DF_M.loc[Cont_DF.index,'INVENTORY']

2018-08-17 10:54:43


In [147]:
HSIPSegNull = Cont_DF.HSIPSegLoc.isnull()
HSIPIntNull = Cont_DF.HSIPIntLoc.isnull()
WPPSNull    = Cont_DF.WPPSLoc.isnull()
EFKNull     = Cont_DF.EFKLoc.isnull()
for i in list(Cont_DF.index):
    if not EFKNull[i]:
        Cont_DF.set_value(i,'IntLoc',Cont_DF.loc[i,'EFKLoc'])
        Cont_DF.set_value(i,'LocSrc','EFK')
    else:
        if not WPPSNull[i]:
            Cont_DF.set_value(i,'SegLoc',Cont_DF.loc[i,'WPPSLoc'])
            Cont_DF.set_value(i,'LocSrc','WPPS')
        else:
            if not HSIPIntNull[i]:
                Cont_DF.set_value(i,'IntLoc',Cont_DF.loc[i,'HSIPIntLoc'])
                Cont_DF.set_value(i,'LocSrc','HSIPInt')
            elif not HSIPSegNull[i]:
                Cont_DF.set_value(i,'SegLoc',Cont_DF.loc[i,'HSIPSegLoc'])
                Cont_DF.set_value(i,'LocSrc','HSIPSeg')
            else:
                Cont_DF.set_value(i,'LocSrc','Unknown')
Cont_DF.LocSrc.value_counts()

WPPS       514
Unknown    209
EFK        120
HSIPSeg     34
HSIPInt     28
Name: LocSrc, dtype: int64

In [155]:
hsmpy3.il.GeocodeContracts(Cont_DF,Project_GDB,'WPPS_Int','WPPS_Seg')
print(arcpy.management.GetCount(Project_GDB+'\\WPPS_Seg')[0])
print(arcpy.management.GetCount(Project_GDB+'\\WPPS_Int')[0])

1043
341


In [9]:
# Add PayItems to Main GDB
print(strftime("%Y-%m-%d %H:%M:%S"))
Main_GDB = AnalysisDir + '\\WPPS_Geocoded.gdb'
XLSList = [os.path.join(Data_Dir, f) for f in os.listdir(Data_Dir) if os.path.isfile(os.path.join(Data_Dir, f)) and f.split('.')[1]=='xls' and len(f.split('.')[0])==5]    
Pay_Table = hsmpy3.il.AddPayItemTabletoGDB(Main_GDB,XLSList,Main_GDB + '\\PayItems')

2018-06-18 15:26:53


In [10]:
# Add HSIP Table to Main GDB
print(strftime("%Y-%m-%d %H:%M:%S"))
Main_GDB = AnalysisDir + '\\WPPS_Geocoded.gdb'
HSIP_Table = hsmpy3.il.AddHSIPTableToGDB(Main_GDB,HSIP_Joined,Main_GDB + '\\HSIP_Applications')

2018-06-18 15:27:45


In [11]:
# Add Contract Table to Main GDB
print(strftime("%Y-%m-%d %H:%M:%S"))
Main_GDB = AnalysisDir + '\\WPPS_Geocoded.gdb'
Cont_Table = hsmpy3.il.AddContractTableToGDB(Main_GDB,Cont_DF,Main_GDB + '\\Contracts')

2018-06-18 15:28:02


In [157]:
# Create Relationships
print(strftime("%Y-%m-%d %H:%M:%S"))
Cont_Table  = Project_GDB + '\\Contracts'
HSIP_Table  = Project_GDB + '\\HSIP_Applications'
Pay_Table   = Project_GDB + '\\PayItems'
SegFC       = Project_GDB + '\\WPPS_Seg'
IntFC       = Project_GDB + '\\WPPS_Int'
R_Cont_HSIP = Project_GDB + '\\Rel_Cont_HSIP'
R_Cont_Pay  = Project_GDB + '\\Rel_Cont_Pay'
R_Cont_Seg  = Project_GDB + '\\Rel_Cont_Seg'
R_Cont_Int  = Project_GDB + '\\Rel_Cont_Int'

arcpy.Delete_management(R_Cont_HSIP)
arcpy.management.CreateRelationshipClass(
    origin_table           = Cont_Table,
    destination_table      = HSIP_Table, 
    out_relationship_class = R_Cont_HSIP, 
    relationship_type      = "SIMPLE", 
    forward_label          = "HSIP_Applications", 
    backward_label         = "Contracts", 
    message_direction      = "NONE", 
    cardinality            = "MANY_TO_MANY", 
    attributed             = "NONE", 
    origin_primary_key     = "ContNum", 
    origin_foreign_key     = "ContNum", 
    destination_primary_key= "HSIPID", 
    destination_foreign_key= 'HSIPID'
)
Cont_HSIP = pd.DataFrame(columns=['ContNum','HSIPID'])
j = 0
for i,r in Cont_DF.iterrows():
    hsip_ids = r.HSIPIDs.split(';')
    for hsip_id in hsip_ids:
        Cont_HSIP.loc[j] = [r.ContNum,hsip_id]
        j += 1
uc = arcpy.InsertCursor(R_Cont_HSIP)
for i,r in Cont_HSIP.iterrows():
    row = uc.newRow()
    row.setValue('ContNum',r.ContNum)
    row.setValue('HSIPID',r.HSIPID)
    uc.insertRow(row)
del uc
    
arcpy.Delete_management(R_Cont_Seg)
arcpy.management.CreateRelationshipClass(
    origin_table           = Cont_Table,
    destination_table      = SegFC, 
    out_relationship_class = R_Cont_Seg, 
    relationship_type      = "SIMPLE", 
    forward_label          = "Segments", 
    backward_label         = "Contracts", 
    message_direction      = "NONE", 
    cardinality            = "ONE_TO_MANY", 
    attributed             = "NONE", 
    origin_primary_key     = "ContNum", 
    origin_foreign_key     = "ContNum", 
    destination_primary_key= "ContNum", 
    destination_foreign_key= 'ContNum'
)
arcpy.Delete_management(R_Cont_Int)
arcpy.management.CreateRelationshipClass(
    origin_table           = Cont_Table,
    destination_table      = IntFC, 
    out_relationship_class = R_Cont_Int, 
    relationship_type      = "SIMPLE", 
    forward_label          = "Intersections", 
    backward_label         = "Contracts", 
    message_direction      = "NONE", 
    cardinality            = "ONE_TO_MANY", 
    attributed             = "NONE", 
    origin_primary_key     = "ContNum", 
    origin_foreign_key     = "ContNum", 
    destination_primary_key= "ContNum", 
    destination_foreign_key= 'ContNum'
)

arcpy.Delete_management(R_Cont_Pay)
arcpy.management.CreateRelationshipClass(
    origin_table           = Cont_Table,
    destination_table      = Pay_Table, 
    out_relationship_class = R_Cont_Pay, 
    relationship_type      = "SIMPLE", 
    forward_label          = "PayItems", 
    backward_label         = "Contracts", 
    message_direction      = "NONE", 
    cardinality            = "ONE_TO_MANY", 
    attributed             = "NONE", 
    origin_primary_key     = "ContNum", 
    origin_foreign_key     = "ContNum", 
    destination_primary_key= "ContNum", 
    destination_foreign_key= 'ContNum'
)
print(strftime("%Y-%m-%d %H:%M:%S"))

2018-08-17 11:00:31


In [14]:
#Update PayItems
#df = hsmpy3.common.FCtoDF(Project_GDB + '\\PayItems')
#Pay_DF = pd.read_csv(PythonDir + '\\Contract_PayItems.csv',index_col=[0,1])
Pay_DF = Pay_DF[~pd.isnull(Pay_DF.PAY_ITEM_CODE)]
uc = arcpy.UpdateCursor(Project_GDB + '\\PayItems')
for row in uc:
    contnum = row.getValue('ContNum')
    payitem = row.getValue('PAY_ITEM')
    paycode = row.getValue('Name')
    if len(paycode)==0 and (contnum,payitem) in Pay_DF.index:
        paycode = Pay_DF.loc[(contnum,payitem),'PAY_ITEM_CODE']
        if len(paycode)>0:
            print(contnum,payitem,paycode)
            row.setValue('Name',paycode)
            uc.updateRow(row)
del row
del uc

99495 40600100 BIT MATLS PR CT
99495 40600300 AGG PR CT
99463 48203029 HMA SHOULDERS   8                    S
99463 64200105 SHOULDER RUMBLE STRIP
99463 78001110 PAINT PVT MK LINE  4
99463 78100300 REPLACEMENT REFLECTOR
99463 67100100 MOBILIZATION                         L
99463 72000100 SIGN PANEL T1                        S
99463 72800100 TELES STL SIN SUPPORT
99443 48203029 HMA SHOULDERS   8                    S
99443 64200105 SHOULDER RUMBLE STRIP
99443 78001110 PAINT PVT MK LINE  4
99443 78000200 THPL PVT MK LINE  4
99443 67100100 MOBILIZATION                         L
99443 78100300 REPLACEMENT REFLECTOR
99443 78000600 THPL PVT MK LINE 12
99443 78000100 THPL PVT MK LTR & SYM                S
99443 78000650 THPL PVT MK LINE 24
99413 63300900 VERT ADJ OF GUARD RL
99413 64200105 SHOULDER RUMBLE STRIP
99413 44000198 HMA SURF REM VAR DP
99413 40600100 BIT MATLS PR CT
99413 40600300 AGG PR CT
99369 48203029 HMA SHOULDERS   8                    S
99369 72000100 SIGN PANEL T1            

93443 X0323153 EC C GROUND  6 1C GRN
93443 60238610 INLETS TA SALV GRATE
93443 25000920 SEEDING CL  1A SPL
93443 81013000 CON T 4 PVC
93443 44003900 MED SURF REM & REPL
93443 40600100 BIT MATLS PR CT
93443 81012800 CON T 3 PVC
93443 81012600 CON T 2 PVC
93443 81012500 CON T 1 1/2 PVC
93443 81012900 CON T 3 1/2 PVC
93443 XX003503 FLARED END SEC REM
93443 81012700 CON T 2 1/2 PVC
93443 70301000 WORK ZONE PAVT MK REM                S
93443 40600300 AGG PR CT
93423 54020804 PCBC   8X4 (M273)
93423 42001400 BR APPROACH PAVT SPL
93423 X0320872 VIDEO VEH DET SYS
93423 54021003 PCBC  10X3 (M273)
93423 70103700 TRAF CONT COMPL
93423 54000900 BOX CUL END SECT SPL
93423 54206739 P CUL 1 RC EQV R-S 24
93423 63000000 SPBGR TY A
93423 54200643 P CUL 1 CS/A CP 18
93423 40600100 BIT MATLS PR CT
93423 54200640 P CUL 1 CS/A CP 15
93423 28000300 TEMP DITCH CHECKS
93423 54200439 P CUL 1 RCCP  24
93423 81021540 CON AUGERED 1 1/2 PVC
93423 42001430 BR APPR PVT CON (FLX)                S
93423 60600097 CLASS

87573 X4060110 BIT MATLS PR CT
87573 78300100 PAVT MARKING REMOVAL                 S
87573 78201000 TERMINAL MARKER - DA
87573 78200410 GUARDRAIL MKR TYPE A
87573 70301000 WORK ZONE PAVT MK REM                S
87573 78200300 PRISMATIC CURB REFL
87546 54260311 TRAVERS PIPE GRATE
87546 78200300 PRISMATIC CURB REFL
87519 40600100 BIT MATLS PR CT
87518 XX008080 STEEL RAIL TY 2399 SP
87518 78201000 TERMINAL MARKER - DA
87518 78200410 GUARDRAIL MKR TYPE A
87518 78200430 GUARDRAIL MKR TYPE C
87517 40600625 LEV BIND MM N50
87517 40603310 HMA SC "C"   N50
87517 48101200 AGGREGATE SHLDS B
87517 40800050 INCIDENTAL HMA SURF
87517 67100100 MOBILIZATION                         L
87517 40600982 HMA SURF REM BUTT JT                 S
87517 78001110 PAINT PVT MK LINE  4
87517 70101830 TRAF CONT-PROT BLR 21                L
87517 40600100 BIT MATLS PR CT
87514 70301000 WORK ZONE PAVT MK REM                S
87514 40600115 P BIT MATLS PR CT
87514 40600300 AGG PR CT
87479 40600100 BIT MATLS PR CT
87479 

78048 Z0030350 IMP ATTN REL NRD  TL3
78047 40603245 P HMA BC IL19.0  N105
78047 40603575 P HMA SC "E"   N105
78047 42001200 PAVEMENT FABRIC                      S
78047 Z0017100 DOWEL BARS
78047 28000300 TEMP DITCH CHECKS
78047 40600115 P BIT MATLS PR CT
78047 70301000 WORK ZONE PAVT MK REM                S
78047 78300105 PAVT MARKING REMOVAL
78045 Z0030150 IMPACT ATTEN NRD  TL3
78045 64200105 SHOULDER RUMBLE STRIP
78045 25001010 SEEDING CL  2 MOD
78045 28000300 TEMP DITCH CHECKS
78042 63000000 SPBGR TY A
78042 63300725 SPBGR (SHORT RADIUS)
78042 63000025 SPBGR ATTACH TO STR
78042 78200405 GUARDRAIL MARKERS
78042 78201000 TERMINAL MARKER - DA
78042 28000300 TEMP DITCH CHECKS
78042 25001010 SEEDING CL  2 MOD
78038 54020503 PCBC   5X3 (M273)
78038 63000000 SPBGR TY A
78038 64200105 SHOULDER RUMBLE STRIP
78038 54001000 BOX CUL END SECT
78038 70101805 TRAF CONT & PROT SPL
78038 Z0030250 IMP ATTN TEMP NRD TL3
78038 54020604 PCBC   6X4 (M273)
78038 54020402 PCBC   4X2 (M273)
78038 70106800 C

76D14 78100105 RAISED REF PVT MKR BR
76D14 Z0029999 IMPACT ATTENUATOR REM
76D14 78200410 GUARDRAIL MKR TYPE A
76D14 Z0030260 IMP ATTN TEMP FRN TL3
76D04 X0323644 PAVT MKING GROOVING
76D04 70301000 WORK ZONE PAVT MK REM                S
76C64 X7015015 TRUSS MTD CHG MSG SGN
76C64 86300305 CONT CAB TYPE III SPL
76C64 73305000 OVHD SIN STR WALKWAY
76C64 X0325470 CON AT ST 3 ALUM
76C64 87800210 CONC FDN TY D SPL
76C64 81012500 CON T 1 1/2 PVC
76C64 X8100065 CON T 4 PVC TY C
76C64 X0323228 F & I TRUSS DAMPER
76C64 81900200 TR & BKFIL F ELECT WK
76C64 81012800 CON T 3 PVC
76C32 48203029 HMA SHOULDERS   8                    S
76C32 64200105 SHOULDER RUMBLE STRIP
76C32 20200100 EARTH EXCAVATION                     C
76C32 67100100 MOBILIZATION                         L
76C32 70100460 TRAF CONT-PROT 701306                L
76C32 40201000 AGGREGATE-TEMP ACCESS
76C32 48102100 AGG WEDGE SHLD TYPE B
76C31 63000000 SPBGR TY A
76C31 63100201 TRAF BAR TERM T5 SPL
76C31 Z0030150 IMPACT ATTEN NRD  TL3
76

...

60T97 78300100 PAVT MARKING REMOVAL                 S
60T95 70106800 CHANGEABLE MESSAGE SN               CA
60T95 78300100 PAVT MARKING REMOVAL                 S
60T94 78300100 PAVT MARKING REMOVAL                 S
60T92 70301000 WORK ZONE PAVT MK REM                S
60T92 40600200 BIT MATLS PR CT
60T92 40600300 AGG PR CT
60T92 40600895 CONSTRUC TEST STRIP
60T92 70106800 CHANGEABLE MESSAGE SN               CA
60T90 78300100 PAVT MARKING REMOVAL                 S
60T88 40600200 BIT MATLS PR CT
60T88 60107700 PIPE UNDERDRAINS  6
60T88 78300100 PAVT MARKING REMOVAL                 S
60T88 40600300 AGG PR CT
60T88 40600895 CONSTRUC TEST STRIP
60T80 78300100 PAVT MARKING REMOVAL                 S
60T79 70106800 CHANGEABLE MESSAGE SN               CA
60T79 78300100 PAVT MARKING REMOVAL                 S
60T50 70301000 WORK ZONE PAVT MK REM                S
60T50 78300100 PAVT MARKING REMOVAL                 S
60T50 40600200 BIT MATLS PR CT
60T50 78200410 GUARDRAIL MKR TYPE A
60T50 78201000

60K24 Z0004562 COMB C C&G REM & REPL
60K24 81018500 CON P 2 GALVS
60K24 81018900 CON P 4 GALVS
60K24 81000600 CON T 2 GALVS
60K24 X8710020 FOCC62.5/125 MM12SM12
60K24 81001000 CON T 4 GALVS
60K24 X8050015 SERV INSTALL POLE MT
60K24 78300400 TH-PL PAVT MK REMOV
60K24 81000800 CON T 3 GALVS
60K24 Z0033090 ELCBL C TRACER 14 1C
60K24 81000700 CON T 2 1/2 GALVS
60K24 81030000 CLEAN EX CONDUIT
60K24 X8730027 ELCBL C GROUND  6  1C
60K24 78300100 PAVT MARKING REMOVAL                 S
60K24 81900200 TR & BKFIL F ELECT WK
60K24 81000900 CON T 3 1/2 GALVS
60K23 85700205 FAC T4 CAB SPL
60K23 X8620020 UNINTER POWER SUPPLY
60K23 Z0004562 COMB C C&G REM & REPL
60K23 81018900 CON P 4 GALVS
60K23 X8710020 FOCC62.5/125 MM12SM12
60K23 81000600 CON T 2 GALVS
60K23 81018500 CON P 2 GALVS
60K23 81001000 CON T 4 GALVS
60K23 81030000 CLEAN EX CONDUIT
60K23 X0324232 PT NEW MAP < 40
60K23 X8050015 SERV INSTALL POLE MT
60K23 Z0033090 ELCBL C TRACER 14 1C
60K23 81000800 CON T 3 GALVS
60K23 85700505 FAC IN EXIST 

60H72 28000255 TEMP EROS CONTR SEED
60H72 54201279 P CUL 2 RCCP  24
60G44 X8620020 UNINTER POWER SUPPLY
60G44 85700205 FAC T4 CAB SPL
60G44 85700505 FAC IN EXIST CAB SPL
60G44 81000700 CON T 2 1/2 GALVS
60G44 X8730027 ELCBL C GROUND  6  1C
60G44 81900200 TR & BKFIL F ELECT WK
60G44 78300400 TH-PL PAVT MK REMOV
60G44 X0322256 TEMP INFO SIGNING
60G44 X0324183 CLEAN EX ELECT MAN/HH
60G44 X8050015 SERV INSTALL POLE MT
60G44 X8950200 REBUILD EX HANDHOLE
60G43 X8620020 UNINTER POWER SUPPLY
60G43 85700205 FAC T4 CAB SPL
60G43 88030030 SH LED 1F 3S SWM
60G43 88030120 SH LED 1F 5S SWM
60G43 85700505 FAC IN EXIST CAB SPL
60G43 81018700 CON P 3 GALVS
60G43 81000600 CON T 2 GALVS
60G43 81018900 CON P 4 GALVS
60G43 81000700 CON T 2 1/2 GALVS
60G43 X8050015 SERV INSTALL POLE MT
60G43 X8730027 ELCBL C GROUND  6  1C
60G43 X8950200 REBUILD EX HANDHOLE
60G43 X0322256 TEMP INFO SIGNING
60G43 81900200 TR & BKFIL F ELECT WK
60G43 81001000 CON T 4 GALVS
60G43 X0320604 PAVT REPLACEMENT BIT
60G42 X8620020 UNI

60F77 63000000 SPBGR TY A
60F77 Z0030150 IMPACT ATTEN NRD  TL3
60F77 64200105 SHOULDER RUMBLE STRIP
60F77 20201006 GRADING & SHAP SHLDS
60F77 Z0002005 ATTENUATOR BASE
60F77 Z0030030 IMP ATTEN FRD NAR TL3
60F77 78200410 GUARDRAIL MKR TYPE A
60F77 78201000 TERMINAL MARKER - DA
60F77 Z0029999 IMPACT ATTENUATOR REM
60F76 64200105 SHOULDER RUMBLE STRIP
60F75 X4060826 P LB MM IL-4.75  N50
60F75 Z0030030 IMP ATTEN FRD NAR TL3
60F75 X8620020 UNINTER POWER SUPPLY
60F75 64200105 SHOULDER RUMBLE STRIP
60F75 Z0002005 ATTENUATOR BASE
60F75 Z0030150 IMPACT ATTEN NRD  TL3
60F75 81000700 CON T 2 1/2 GALVS
60F75 78201000 TERMINAL MARKER - DA
60F75 81900200 TR & BKFIL F ELECT WK
60F75 X8050015 SERV INSTALL POLE MT
60F75 X8140074 GROUND HH FR & COVER
60F75 X8730027 ELCBL C GROUND  6  1C
60F75 X8950210 REBLD EX HH TO HD HH
60F75 40600895 CONSTRUC TEST STRIP
60F75 78200410 GUARDRAIL MKR TYPE A
60F75 87800115 CONC FDN TY A
60F74 64200105 SHOULDER RUMBLE STRIP
60F74 63000000 SPBGR TY A
60F74 20201006 GRADING

60D34 X8160395 UD 3#2 #4G EPRRHW1.5
60D34 83600400 POLE FOUNDATION METAL
60D34 X0325590 HT CBL MED BAR TERM
60D34 81018700 CON P 3 GALVS
60D34 81701385 EC C EPR USE 3-1C 350
60D34 81000800 CON T 3 GALVS
60D34 64200105 SHOULDER RUMBLE STRIP
60D34 X0323574 MAINTAIN LIGHTING SYS
60D34 81900200 TR & BKFIL F ELECT WK
60D34 XX006937 GROUND ROD 5/8 X 10
60D34 X0325606 HT CBL MED BAR DEMO
60D34 63000000 SPBGR TY A
60D34 X0322256 TEMP INFO SIGNING
60D33 X0325589 HT CBL MEDIAN BARRIER
60D33 X0325590 HT CBL MED BAR TERM
60D33 X0325606 HT CBL MED BAR DEMO
60D32 X0325589 HT CBL MEDIAN BARRIER
60D32 X0325590 HT CBL MED BAR TERM
60D32 X0325606 HT CBL MED BAR DEMO
60D20 81018500 CON P 2 GALVS
60D20 X8710020 FOCC62.5/125 MM12SM12
60D20 X0324007 OPTIM TRAF SIGNAL SYS
60D20 X0322925 ELCBL C TRACER 14 1C
60D20 X0324773 FLASH BEACON INSTALL
60D20 88030030 SH LED 1F 3S SWM
60D20 X8620020 UNINTER POWER SUPPLY
60D20 70106800 CHANGEABLE MESSAGE SN               CA
60D20 88030120 SH LED 1F 5S SWM
60D20 X8050015